In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR

In [30]:
# reading the data
df = pd.read_csv(r"..\data\AAPL Stock Price (15-06-204 and Last 30 days).csv", 
                 parse_dates=['Date'], dayfirst=True)
df.columns = df.columns.str.lower()

In [31]:
display(df.head(),df.tail(),df.sample(5))

,date,open,high,low,close,volume
0,2024-05-16 16:00:00,190.47,191.10,189.66,189.84,52845230
1,2024-05-17 16:00:00,189.51,190.81,189.18,189.87,41282925
2,2024-05-20 16:00:00,189.35,191.92,189.01,191.04,44361275
3,2024-05-21 16:00:00,191.09,192.73,190.92,192.35,42309401
4,2024-05-22 16:00:00,192.27,192.82,190.27,190.90,34648547


,date,open,high,low,close,volume
16,2024-06-10 16:00:00,196.90,197.30,192.15,193.12,97262077
17,2024-06-11 16:00:00,193.65,207.16,193.63,207.15,172373296
18,2024-06-12 16:00:00,207.37,220.20,206.90,213.07,198134293
19,2024-06-13 16:00:00,214.74,216.75,211.60,214.24,97862729
20,2024-06-14 16:00:00,213.85,215.17,211.30,212.49,70122748


,date,open,high,low,close,volume
2,2024-05-20 16:00:00,189.35,191.92,189.01,191.04,44361275
15,2024-06-07 16:00:00,194.65,196.94,194.14,196.89,53103912
14,2024-06-06 16:00:00,195.69,196.50,194.17,194.48,41181753
5,2024-05-23 16:00:00,190.98,191.00,186.63,186.88,51005924
3,2024-05-21 16:00:00,191.09,192.73,190.92,192.35,42309401


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    21 non-null     datetime64[ns]
 1   open    21 non-null     float64       
 2   high    21 non-null     float64       
 3   low     21 non-null     float64       
 4   close   21 non-null     float64       
 5   volume  21 non-null     int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 1.1 KB


In [37]:
# creating three different support vector machines
linear_svr = SVR(kernel='linear', C=1000)
polynomial_svr = SVR(kernel='poly', C=1000, degree=2)
radial_basis_function_svr = SVR(kernel='rbf', C=1000, gamma=0.1)